In [1]:
from pyspark.find_spark_home import _find_spark_home
from pyspark.sql import SparkSession
from pyspark import SparkConf, StorageLevel
from pyspark.sql.functions import col, split, expr
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType
import os

from dotenv import load_dotenv
load_dotenv()

print(_find_spark_home())

C:\Users\liisu\anaconda3\envs\bigData\Lib\site-packages\pyspark


In [2]:
python_path = os.environ.get('PYTHON_PATH')
app_name_nov = os.environ.get('APP_NAME_NOV')
hadoop_path_nov = os.environ.get('HADOOP_NOV_DATASET_PATH')

In [3]:
conf = SparkConf()\
    .setMaster('local[*]')\
    .set('spark-local-dir', "C:\\spark-temp")\
    .set('spark.driver.memory', '4g')\
    .set('spark.executor.memory', '4g')\
    .set('spark.driver.maxResultSize', '2g')\
    .set('spark.pyspark.python', python_path)\
    .set('spark.pyspark.driver.python', python_path)\
    .set("spark.network.timeout","800s")\
    .set("spark.dynamicAllocation.enabled", "true")\
    .set("spark.shuffle.service.enabled", "true")\
    .set("spark.dynamicAllocation.minExecutors", "1")\
    .set("spark.dynamicAllocation.maxExecutors", "10")\
    .set("spark.dynamicAllocation.executorIdleTimeout", "60s")\

spark = SparkSession.builder.appName(app_name_nov).config(conf=conf).getOrCreate()
sc = spark.sparkContext

for item in sc.getConf().getAll(): print(item)

('spark.driver.host', 'host.docker.internal')
('spark.dynamicAllocation.minExecutors', '1')
('spark.app.id', 'local-1703870983092')
('spark.shuffle.service.enabled', 'true')
('spark.driver.memory', '4g')
('spark.executor.memory', '4g')
('spark.dynamicAllocation.maxExecutors', '10')
('spark.executor.id', 'driver')
('spark.pyspark.driver.python', 'C:/Users/liisu/anaconda3/envs/bigData/python.exe')
('spark.app.submitTime', '1703870980181')
('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.b

In [4]:
df = spark.read.csv(hadoop_path_nov, header = True, inferSchema = True)
df.show()

+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|2019-11-01 01:00:00|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 01:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|  janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 01:00:01|      view|  17302664|2053013553853497655|                null|   creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 01:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|      lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 01:00:01|      view|   1004775|2053013555631882655|electronics.s

In [5]:
df_index = df.select('*').withColumn('id', F.monotonically_increasing_id())
column_names = ['id'] + [col for col in df.columns]

df_index_first = df_index.select(column_names)
df_index_first.show()

+---+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
| id|         event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+---+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|  0|2019-11-01 01:00:00|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|  1|2019-11-01 01:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|  janome|293.65|530496790|8e5f4f83-366c-4f7...|
|  2|2019-11-01 01:00:01|      view|  17302664|2053013553853497655|                null|   creed| 28.31|561587266|755422e7-9040-477...|
|  3|2019-11-01 01:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|      lg|712.87|518085591|3bfb58cd-7892-48c...|
|  4|2019-11-01 01:00:01|      view|   1004775|2

In [6]:
df_new = df_index_first.drop('user_session', 'category_id')
df_new.show()

+---+-------------------+----------+----------+--------------------+--------+------+---------+
| id|         event_time|event_type|product_id|       category_code|   brand| price|  user_id|
+---+-------------------+----------+----------+--------------------+--------+------+---------+
|  0|2019-11-01 01:00:00|      view|   1003461|electronics.smart...|  xiaomi|489.07|520088904|
|  1|2019-11-01 01:00:00|      view|   5000088|appliances.sewing...|  janome|293.65|530496790|
|  2|2019-11-01 01:00:01|      view|  17302664|                null|   creed| 28.31|561587266|
|  3|2019-11-01 01:00:01|      view|   3601530|appliances.kitche...|      lg|712.87|518085591|
|  4|2019-11-01 01:00:01|      view|   1004775|electronics.smart...|  xiaomi|183.27|558856683|
|  5|2019-11-01 01:00:01|      view|   1306894|  computers.notebook|      hp|360.09|520772685|
|  6|2019-11-01 01:00:01|      view|   1306421|  computers.notebook|      hp|514.56|514028527|
|  7|2019-11-01 01:00:02|      view|  15900065|   

In [7]:
df_new = df_new.withColumn("split_code", split(col("category_code"), "\."))

df_new = df_new.withColumn("category", col("split_code")[0])

df_new = df_new.withColumn("product", expr("substring(category_code, length(category) + 2)"))

df_new =df_new.drop("split_code")

df_new.show(truncate=False)

+---+-------------------+----------+----------+--------------------------------+--------+------+---------+-----------+---------------------+
|id |event_time         |event_type|product_id|category_code                   |brand   |price |user_id  |category   |product              |
+---+-------------------+----------+----------+--------------------------------+--------+------+---------+-----------+---------------------+
|0  |2019-11-01 01:00:00|view      |1003461   |electronics.smartphone          |xiaomi  |489.07|520088904|electronics|smartphone           |
|1  |2019-11-01 01:00:00|view      |5000088   |appliances.sewing_machine       |janome  |293.65|530496790|appliances |sewing_machine       |
|2  |2019-11-01 01:00:01|view      |17302664  |null                            |creed   |28.31 |561587266|null       |null                 |
|3  |2019-11-01 01:00:01|view      |3601530   |appliances.kitchen.washer       |lg      |712.87|518085591|appliances |kitchen.washer       |
|4  |2019-11-

In [8]:
df_new =df_new.drop("category_code")
df_new.show(truncate=False)

+---+-------------------+----------+----------+--------+------+---------+-----------+---------------------+
|id |event_time         |event_type|product_id|brand   |price |user_id  |category   |product              |
+---+-------------------+----------+----------+--------+------+---------+-----------+---------------------+
|0  |2019-11-01 01:00:00|view      |1003461   |xiaomi  |489.07|520088904|electronics|smartphone           |
|1  |2019-11-01 01:00:00|view      |5000088   |janome  |293.65|530496790|appliances |sewing_machine       |
|2  |2019-11-01 01:00:01|view      |17302664  |creed   |28.31 |561587266|null       |null                 |
|3  |2019-11-01 01:00:01|view      |3601530   |lg      |712.87|518085591|appliances |kitchen.washer       |
|4  |2019-11-01 01:00:01|view      |1004775   |xiaomi  |183.27|558856683|electronics|smartphone           |
|5  |2019-11-01 01:00:01|view      |1306894   |hp      |360.09|520772685|computers  |notebook             |
|6  |2019-11-01 01:00:01|vie

In [9]:
df_new = df_new.withColumn("date", split(col("event_time"), " ")[0])
df_new = df_new.withColumn("day", split(col("date"), "-")[2].cast(IntegerType()))
df_new = df_new.drop("event_time")

df_new.show()

+---+----------+----------+--------+------+---------+-----------+--------------------+----------+---+
| id|event_type|product_id|   brand| price|  user_id|   category|             product|      date|day|
+---+----------+----------+--------+------+---------+-----------+--------------------+----------+---+
|  0|      view|   1003461|  xiaomi|489.07|520088904|electronics|          smartphone|2019-11-01|  1|
|  1|      view|   5000088|  janome|293.65|530496790| appliances|      sewing_machine|2019-11-01|  1|
|  2|      view|  17302664|   creed| 28.31|561587266|       null|                null|2019-11-01|  1|
|  3|      view|   3601530|      lg|712.87|518085591| appliances|      kitchen.washer|2019-11-01|  1|
|  4|      view|   1004775|  xiaomi|183.27|558856683|electronics|          smartphone|2019-11-01|  1|
|  5|      view|   1306894|      hp|360.09|520772685|  computers|            notebook|2019-11-01|  1|
|  6|      view|   1306421|      hp|514.56|514028527|  computers|            noteb

In [10]:
df_new = df_new.drop("date")
df_new.show()

+---+----------+----------+--------+------+---------+-----------+--------------------+---+
| id|event_type|product_id|   brand| price|  user_id|   category|             product|day|
+---+----------+----------+--------+------+---------+-----------+--------------------+---+
|  0|      view|   1003461|  xiaomi|489.07|520088904|electronics|          smartphone|  1|
|  1|      view|   5000088|  janome|293.65|530496790| appliances|      sewing_machine|  1|
|  2|      view|  17302664|   creed| 28.31|561587266|       null|                null|  1|
|  3|      view|   3601530|      lg|712.87|518085591| appliances|      kitchen.washer|  1|
|  4|      view|   1004775|  xiaomi|183.27|558856683|electronics|          smartphone|  1|
|  5|      view|   1306894|      hp|360.09|520772685|  computers|            notebook|  1|
|  6|      view|   1306421|      hp|514.56|514028527|  computers|            notebook|  1|
|  7|      view|  15900065| rondell| 30.86|518574284|       null|                null|  1|

In [11]:
df_new = df_new.dropDuplicates()

In [12]:
df_new = df_new.dropna()
df_new.show()

+----+----------+----------+--------+-------+---------+-----------+--------------------+---+
|  id|event_type|product_id|   brand|  price|  user_id|   category|             product|day|
+----+----------+----------+--------+-------+---------+-----------+--------------------+---+
|  29|      view|   1307340|    asus| 360.34|520772685|  computers|            notebook|  1|
| 878|      view|   3100640| redmond|  51.22|522761204| appliances|     kitchen.blender|  1|
|1401|      view|   1004158| samsung| 733.61|560865203|electronics|          smartphone|  1|
|1505|      view|   1201362|    acer|  178.9|513468656|electronics|              tablet|  1|
|2051|      view|   1004777|  xiaomi| 135.01|566282204|electronics|          smartphone|  1|
|4046|      view|   1004386|    oppo| 776.08|512521821|electronics|          smartphone|  1|
|4971|      view|  45500010|deepcool|  17.99|513569912|  computers|            notebook|  1|
|5170|      view|   1480163|    acer| 617.75|566284428|  computers|   

In [13]:
print("Null values present in:")
for c in ["category","product","day", "brand"]:
    print(c +':', df_new.where(F.col(c).isNull()).count())

Null values present in:
category: 0
product: 0
day: 0
brand: 0


In [14]:
count = df_new.count()
print(f"Total number of rows: {count:,d}")

Total number of rows: 42,089,570


In [15]:
df_new.describe().show()

+-------+--------------------+----------+--------------------+--------+------------------+--------------------+-----------+-----------------+------------------+
|summary|                  id|event_type|          product_id|   brand|             price|             user_id|   category|          product|               day|
+-------+--------------------+----------+--------------------+--------+------------------+--------------------+-----------+-----------------+------------------+
|  count|            42089570|  42089570|            42089570|42089570|          42089570|            42089570|   42089570|         42089570|          42089570|
|   mean|2.843057495002189...|      null|   7069141.265143645|Infinity|360.21407357502267| 5.389993519299238E8|       null|             null|15.558973042490098|
| stddev|1.660905644135284...|      null|1.4095484819486292E7|     NaN| 383.3233963131343|2.2914563921821624E7|       null|             null|7.4815895100845555|
|    min|                   0|    

In [16]:
save_path = hadoop_path_nov + '/selected_Data/' 

df_new.repartition(8).write.mode('overwrite').option('header','true').csv(save_path)
spark.stop()